**Scrape YT for Language data, News, talkshow etc**



##Import Dependencies

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [ ]:
#Import the necessary modules
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sagemaker, boto3, json
from googleapiclient.discovery import build
from pytube import YouTube
from pytube.innertube import _default_clients
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"] #fixes the age restriction error

from tqdm import tqdm
import time

# Get Video_id from the Channels


##Create an Instance

In [ ]:
api_key = " "  #insert your API key

#Use "https://commentpicker.com/youtube-channel-id.php" to get channel ID
#channel_id = "UCMvNXvXnDUuj7NC6Gaz7paw" #EAYoruba
channel_id = "UCwsfVP4kQZ0iy-LPSgRQ39w" #Yoruba Brodcasting network

youtube = build("youtube", "v3", developerKey=api_key)

##Get channel stats

In [ ]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id)
    response = request.execute()
    data = dict(channel_name = response['items'][0]['snippet']['title'],
                description = response['items'][0]['snippet']['description'],
                total_videos = response['items'][0]['statistics']['videoCount'],
                upload_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    )
    return data

In [ ]:
stats = get_channel_stats(youtube, channel_id)
channel_name = stats.get('channel_name')
channel_name

##DataFrame of Channel data

In [ ]:
channel_data = pd.DataFrame(stats, index=[0])
channel_data

,channel_name,description,total_videos,upload_id
0,YORUBA BROADCASTING NETWORK (YBN),A KI YIN KAABO SI IKAANI YORUBA BROADCASTING N...,1326,UUwsfVP4kQZ0iy-LPSgRQ39w


##Get all Video_id using the Playlist_id

In [ ]:
#Upload id corresponds to playlist id

playlist_id = channel_data['upload_id'][0]
playlist_id


'UUwsfVP4kQZ0iy-LPSgRQ39w'

In [ ]:
def get_video_ids(youtube, playlist_id):
  request = youtube.playlistItems().list(
      part='contentDetails',
      playlistId=playlist_id,
      maxResults=50)
  response = request.execute()

  video_ids = []

  for i in range(len(response['items'])):
    video_ids.append(response['items'][i]['contentDetails']['videoId'])

  next_page_token = response.get('nextPageToken')
  more_pages = True

  while more_pages:
    if next_page_token is None:
      more_pages = False
    else:
      request = youtube.playlistItems().list(
                  part='contentDetails',
                  playlistId=playlist_id,
                  maxResults=50,
                  pageToken= next_page_token)
      response = request.execute()

      for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

      next_page_token = response.get('nextPageToken')

  return video_ids

In [ ]:
vid_id = get_video_ids(youtube, playlist_id)
vid_id[0:10]

['YrX9fRNPEf4',
 'OXg8Q4wMS6Q',
 '984FjIVfCq0',
 'l-cMivvMXBM',
 '7CSj_hIY2c4',
 'jXENawyNF7k',
 '53TgPHNgmno',
 'd4Rjh7BoZ1U',
 'Zsdy_QoAKlw',
 'zUkft2i4obs']

#**Scrape Audio from Youtube Video Using PyTube**

##Get audio from 1 Video url

In [ ]:
#Get audio from 1 url

def get_audio_frm_yt():
  #get the video usrl and load it
  video_link = str(input("Enter link here:  "))
  youtube = YouTube(video_link)

  # Extract audio only from the video
  download = youtube.streams.filter(only_audio=True).first()
  download.download()

  #Point to download directory
  destination = str(input('Enter destination directory here: '))

  #Download the audio file and convert the file extension to mp3
  out_file= download.download(output_path=destination)
  base, ext = os.path.splitext(out_file)
  new_file = base + '.mp3'
  os.rename(out_file, new_file)

  return str(new_file) + " has been successfully downloaded.", new_file

In [ ]:
get_audio_frm_yt()

KeyboardInterrupt: Interrupted by user

##Get Audio from multiple Video urls

1.   Google Colab code
2.   



In [ ]:
#Full URL = base_url >https://www.youtube.com/watch?v= [+] vid_id
vid_id

['yZsjOj7SZKg',
 'RKxC9GDVg-Q',
 'P7TKq5EKOQg',
 'cAISXB7ATBk',
 '_JlW9iA-F2Y',
 '8MlUk1FgQeU',
 'Icq1TB8hT6M',
 '8YuRZhVzHbo',
 'mhm32kbyO6c',
 '7I2psdaR-4o',
 'oos06BlNJDU',
 'lkJrHa8rufQ',
 'bBYyBfu_spQ',
 '6dqPeZH7NMk',
 'HqPtMX652DM',
 'NW8WLv0WYro',
 'eWOLnpS0Xok',
 'rudQmrwwcfY',
 '0mk_KMNuFQ4',
 '9KATmjfyAkc',
 'EEykW1CwMLI',
 'tDbu_Di4cwk',
 'cZ1wBCj7rrQ',
 'A2uHlSZepVg',
 '9mAz2ZPqqrg',
 'VSA1FBjD1x0',
 '4ZlvVoP2Hmo',
 'aveunfcDMsk',
 'hxcHjgzs04k',
 'GytLaAdY9G0',
 '6eXwypTup-U',
 '7UHvmnfPwOY',
 '9x2g0u8pnao',
 '58vBrI-dxa8',
 '_mlY7UgfmiU',
 'pHA4kcPQy54',
 'nYLoVt4wvFY',
 'sy9n0dSfetQ',
 'WU7KQAYKXR0',
 'ZnP1bfSmnY0',
 'nKM-iVQSdps',
 'mQJG5F7xrm4',
 'RjUPraU7zWM',
 'g8CL6FfqmFM',
 '9f2W0jdXycQ',
 'cQKoLvuzVao',
 'GQ2y8GBvITw',
 'iBMJRWtr4mM',
 'BYWXuzcKS-Q',
 'w9ycDFt9hN0',
 'vuseINbRcIo',
 'iASqCsnezT8',
 'cPuidoxag4g',
 '-jQjHgrm__U',
 '3sN149XVFJs',
 'LEUGFsbyIRg',
 'KSCTrkTsxjA']

In [ ]:
def get_audio_frm_n_yt():

#iterate through the list of ids in vid_id to make a full url
  #for i in range(len(vid_id)):
  for i in tqdm(range(len(vid_id)), desc="Downloading Audio"):
    time.sleep(0.5)
    #id = vid_id[i]
    vid_lnk = f'https://www.youtube.com/watch?v={vid_id[i]}'
    youtube = YouTube(vid_lnk)

    # Extract audio only from the video
    download = youtube.streams.filter(only_audio=True).first()
    #download.download() #--Removed this to manage the colab Disk space for Temp Files

    #Point to download directory --- Goolge Drive
    destination = '/content/drive/MyDrive/Datasets/multi_aud/Yoruba/Yoruba Brodcasting Network'

    #Download the audio file and convert the file extension to mp3
    out_file= download.download(output_path=destination)
    base, ext = os.path.splitext(out_file)
    new_file = base + '.mp3'
    os.rename(out_file, new_file)


  return str(new_file) + " has been successfully downloaded."

In [1]:
#get_audio_frm_n_yt()

# Get_audio save to S3 -aws

*   Change Bucket name
*   Run in SageMaker



In [2]:
mybucket = 'test_yt_bart'
prefix ='Youtube_lang_data'

In [3]:
def get_audio_frm_n_yt():

  #iterate through the list of ids in vid_id to make a full url
  for i in tqdm(range(len(vid_id)), desc="Downloading Audio"):
    time.sleep(0.5)
    #id = vid_id[i]
    vid_lnk = f'https://www.youtube.com/watch?v={vid_id[i]}'
    youtube = YouTube(vid_lnk)

    # Extract audio only from the video
    download = youtube.streams.filter(only_audio=True).first()
    #download.download()

    #Point to download directory
    destination = 's3://{}/{}'.format(mybucket, prefix)

    #Save Result to s3
    out_file= download.download(output_path=destination)
    base, ext = os.path.splitext(out_file)
    new_file = f"{os.path.basename(base)}.mp3"
    os.rename(out_file, new_file)

    boto3.Session().resource("s3").Bucket(mybucket).Object(
        os.path.join(prefix, channel_name, new_file)
    ).upload_file(new_file)


  return f"{new_file} has been successfully downloaded."